### **The Following IPython script is for deploying the DeepGlobe Image segmentation models using StreamLit and exposing it to external IP adress using ngrok. All 3 models are used for segmentation, and using an ensemble combination for prediction** 

#### **(1) Installing the necessary packages**

In [ ]:
!pip install streamlit

In [12]:
!pip install pyngrok==4.1.1

     |████████████████████████████████| 746 kB 3.6 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.6-py3-none-any.whl size=19263 sha256=d30e158529d76f845eec70a0829660c1f32a61a595d017485bfa8c38ebcc22e0
  Stored in directory: /root/.cache/pip/wheels/d5/8c/c4/8d9cbca4fa19bf64887b4a91914194bb9033f1a7cbb344d5ab
Successfully built pyngrok


#### **(2) Creating the web app using streamlit based API, and saving it in form of app.py**  

In [128]:
%%writefile app.py

# Installing necessary packages
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image, ImageOps
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# Pixel list is the list of actual 3D pixels on which the class predictions have to be mapped to
pixel_list = [(0,0,0), (255,255,0), (255,0,255), (0,255,0), (0,0,255), (255,255,255), (0,255,255)]

# To Avoid Warning
st.set_option('deprecation.showfileUploaderEncoding', False)

st.title("DeepGlobe Landcover classification")

st.text("Upload an image")

# Storing the Models in Cache memory allows to avoid reloading the model agin and again
@st.cache(allow_output_mutation=True)
def load_models():
  model1 = load_model('/content/drive/MyDrive/vgg16_backbone.hdf5', compile=False)
  model2 = load_model('/content/drive/MyDrive/res34_backbone.hdf5', compile=False)
  model3 = load_model('/content/drive/MyDrive/incep_backbone.hdf5', compile=False)
  return model1, model2, model3

model1, model2, model3 = load_models()

# Defining the Prediction Function for the main segmentation
def predict_from_models(image1, model1, model2, model3):
  size = (512, 512)

  # (1)... Image Preprocessing
  image = ImageOps.fit(image1, size, Image.ANTIALIAS)
  image = np.asarray(image)
  image = image/255.0
  image = image[np.newaxis, ...]
  
  # (2) Model prediction
  predict1 = model1.predict(image)
  predict2 = model2.predict(image)
  predict3 = model3.predict(image)

  # (3) Image Reprocessing
  pred = np.squeeze((0.3 * predict1) + (0.2 * predict2) + (0.2 * predict3), axis=0)
  weighted_ensemble = np.argmax(pred, axis=-1)

  return weighted_ensemble

# Creating an UploadFile object to make predictions
file_up = st.file_uploader("Please Upload a Satellite Image", type=['jpg', 'png'])


if file_up is None:
  st.text("Please upload an image !!") 
else:
  image = Image.open(file_up)
  st.image(image, caption='Uploaded Image')
  st.write("Predicting...")
  output_img = predict_from_models(image, model1, model2, model3)
  
  real_img = np.zeros((512, 512, 3))

  # The Loop below is used to map the class predictions, to their respective pixel predictions
  for i in range(7):
    a, b = np.where(output_img == i)
    pxl = pixel_list[i]
  
    for j in range(len(a)):
      real_img[a[j], b[j]] = np.asarray(pxl)

  # Putting the image as output
  st.image(real_img, clamp=True, channels='RGB', caption = 'Predicted Output')

Overwriting app.py


#### **(3) Using the Ngrok toekn to expose the local host web-app to an external IP address**

In [ ]:
!ngrok authtoken ## Place Your own Authtoken recieved while signing up in ngrok##

In [ ]:
# Running the model on streamlit on port 80
!streamlit run --server.port 80 app.py &>/dev/null&

In [125]:
# Exposing the Port to external IP using Ngrok
from pyngrok import ngrok
public_url = ngrok.connect(port='80')
public_url

'http://a82e1667b7af.ngrok.io'

In [ ]:
# One can use this code if the number of ports requested exceeds the allowed ports in free-tier
!killall ngrok